In [7]:
import flet as ft
import requests

AREA_URL = "http://www.jma.go.jp/bosai/common/const/area.json"
FORECAST_URL = "https://www.jma.go.jp/bosai/forecast/data/forecast/{}.json"


def main(page: ft.Page):
    page.title = "気象庁 天気予報アプリ"
    page.window_width = 900
    page.window_height = 600

    # ---------- 地域リスト取得 ----------
    area_data = requests.get(AREA_URL).json()
    offices = area_data["offices"]  # 地方ごとのデータ

    forecast_text = ft.Text("地域を選択してください", size=18)

    # ---------- 天気取得処理 ----------
    def show_weather(area_code, area_name):
        forecast_text.value = f"{area_name} の天気を取得中..."
        page.update()

        weather_data = requests.get(FORECAST_URL.format(area_code)).json()

        # 今日の天気（簡易）
        today = weather_data[0]
        weather = today["timeSeries"][0]["areas"][0]["weathers"][0]

        forecast_text.value = f"【{area_name}】\n{weather}"
        page.update()

    # ---------- NavigationRail ----------
    nav_items = []

    for office_code, office_info in offices.items():
        nav_items.append(
            ft.NavigationRailDestination(
                icon=ft.Icons.LOCATION_ON,
                label=office_info["name"]
            )
        )

    def on_change(e):
        index = e.control.selected_index
        office_code = list(offices.keys())[index]
        office_name = offices[office_code]["name"]
        show_weather(office_code, office_name)

    navigation_rail = ft.NavigationRail(
        selected_index=None,
        label_type=ft.NavigationRailLabelType.ALL,
        destinations=nav_items,
        on_change=on_change,
        expand=True,
    )

    page.add(
        ft.Row(
            [
                navigation_rail,
                ft.VerticalDivider(width=1),
                ft.Column(
                    [
                        forecast_text
                    ],
                    expand=True,
                )
            ],
            expand=True
        )
    )


await ft.app_async(target=main)
